# video 导入

# 显示切片

In [1]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
from tqdm import tqdm

from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    SpatialPadd,
    RandRotate90d,
    CenterSpatialCropd,
    ResizeWithPadOrCropd,
    Flipd,
    Rotate90d,
    RandAffined,
    RandGaussianNoised,
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import UNETR

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    pad_list_data_collate,
    SmartCacheDataset,
    ArrayDataset,
    Dataset
)

import numpy as np
import torch

In [2]:
import gradio as gr
import matplotlib.pyplot as plt
import torch
import nibabel as nib
import numpy as np
import SimpleITK as sitk

def calculate_volume(mask_image_path):
    # 读取分割结果的图像文件
    mask_image = sitk.ReadImage(mask_image_path)

    # 获取图像的大小、原点和间距
    size = mask_image.GetSize()
    origin = mask_image.GetOrigin()
    spacing = mask_image.GetSpacing()

    # 将 SimpleITK 图像转换为 NumPy 数组
    mask_array = sitk.GetArrayFromImage(mask_image)

    # if len(np.unique(mask_array)) != 5:
    #     print(mask_image_path[-15:-12])
    #     print(np.unique(mask_array))
    
    # 计算非零像素的数量
    one_voxels = (mask_array == 1).sum()
    two_voxels = (mask_array == 2).sum()
    three_voxels = (mask_array == 3).sum()
    four_voxels = (mask_array == 4).sum()
    # print(one_voxels,two_voxels,three_voxels,four_voxels)
    # 计算像素的体积（以立方毫米为单位）
    voxel_volume_mm3 = spacing[0] * spacing[1] * spacing[2]

    # 计算体积（以 mm³ 为单位）
    V_Right_ventricular_cistern = one_voxels * voxel_volume_mm3 / 1000.0
    V_Right_cerebral_sulcus = two_voxels * voxel_volume_mm3 / 1000.0
    V_Left_ventricular_cistern = three_voxels * voxel_volume_mm3 / 1000.0
    V_Left_cerebral_sulcus = four_voxels * voxel_volume_mm3 / 1000.0
    # 如果需要以其他单位（例如 cm³）显示，请进行适当的单位转换
    # volume_cm3 = volume_mm3 / 1000.0

    return size,spacing,V_Right_ventricular_cistern, V_Right_cerebral_sulcus, V_Left_ventricular_cistern, V_Left_cerebral_sulcus
    
def process_nii_file(input_nii_file, slice, mode):
            
    if mode == "Step1:Segment":
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        root_dir = "./run"
        model = UNETR(
            in_channels=1,
            out_channels=5,
            img_size=(96, 96, 16),
            feature_size=16,
            hidden_size=768,
            mlp_dim=3072,
            num_heads=12,
            pos_embed="perceptron",
            norm_name="instance",
            res_block=True,
            dropout_rate=0.0,
        ).to(device)
        model.load_state_dict(torch.load(os.path.join(root_dir, "best_metric_model67v2.pth")))
        
        test_transforms = Compose(
            [
            LoadImaged(keys=["image"]),
            EnsureChannelFirstd(keys=["image"]),
            Orientationd(keys=["image"], axcodes="RAS"),
            ScaleIntensityRanged(
                keys=["image"],
                a_min=-50,
                a_max=100,
                b_min=0.0,
                b_max=1.0,
                clip=True,
            ),
            Rotate90d(keys=["image"], k=1)
            # ResizeWithPadOrCropd(keys=["image"], spatial_size=(512, 512, 16)),
            ]
        )
        test_file = [{'image':input_nii_file.name}]
        test_image = SmartCacheDataset(data=test_file, transform=test_transforms)[0]['image']

        with torch.no_grad():

            inputs = torch.unsqueeze(test_image, 1).cuda()

            val_outputs = sliding_window_inference(inputs, (96, 96, 16), 8, model, overlap=0.8)
        
        # Process the output image
        output_image = torch.argmax(val_outputs, dim=1).detach().cpu().squeeze(0)
        
        # Display the images
        fig1 = plt.figure()
        plt.title("image")
        plt.axis('off') # Remove axis
        plt.imshow(inputs.cpu().numpy()[0, 0, :, :, slice], cmap="gray")

        fig2 = plt.figure()
        plt.title("output")
        plt.axis('off') # Remove axis
        plt.imshow(torch.argmax(val_outputs, dim=1).detach().cpu()[0, :, :, slice])



        val_outputs = torch.argmax(val_outputs, dim=1).detach().cpu()[0, :, :, :]
        val_outputs = val_outputs.numpy().astype('int16')
        # val_outputs = np.transpose(val_outputs, (2, 1, 0))
        val_outputs = np.rot90(val_outputs, k=3)
        val_outputs = nib.Nifti1Image(val_outputs, np.eye(4))
        output_path = f'{input_nii_file.name[-15:-7]}_mask.nii.gz'
        nib.save(val_outputs, output_path)

        return ["指定切片分割结果如下,", output_path, fig1, fig2]
    
    if mode == "Step2:Volumn":
        maskFilePath = input_nii_file.name
        size,spacing,V_Right_ventricular_cistern, V_Right_cerebral_sulcus, V_Left_ventricular_cistern, V_Left_cerebral_sulcus = calculate_volume(maskFilePath)

        vol = f"""右侧脑室脑池的体积为{V_Right_ventricular_cistern}cm³\n 右侧脑沟的体积为{V_Right_cerebral_sulcus}cm³\n 左侧脑室脑池的体积为{V_Left_ventricular_cistern}cm³\n 左侧脑沟的体积为{V_Left_cerebral_sulcus}cm³"""
        fig1 = plt.figure()
        fig2 = plt.figure()
        return [vol, None, fig1, fig2]


# Define the Gradio interface
iface = gr.Interface(fn=process_nii_file,
    inputs= 
    [gr.File(file_count='single', file_types=['.nii.gz']), 
    gr.Slider(0, 24, value=8, label="Select Slice", step=1),
    gr.Radio(
            ["Step1:Segment", "Step2:Volumn"], label="mode"
        ),
    ],
    
    outputs=[gr.Text(label="Output"), gr.File(label="下载mask文件"), gr.Plot(label="image"), gr.Plot(label="mask")],  # Display both "image" and "output"
)

iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f8338d877679ad9d93.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
Loading dataset: 100%|██████████| 1/1 [00:00<00:00,  6.94it/s]
cache_num is greater or equal than dataset length, fall back to regular monai.data.CacheDataset.
Loading dataset: 100%|██████████| 1/1 [00:00<00:00,  8.53it/s]
